**Name:** WANG Enrui

**EID:** enruiwang2

# CS4487 - Tutorial 5: Predicting Popularity of Online News

In this tutorial you will train regression models to predict the number of "shares" of a news article on Mashable.

First we need to initialize Python.  Run the below cell.

In [ ]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
import glob
import os
import csv
import string
random.seed(100)


## 1. Loading Data and Pre-processing
Next we need to load the data.  Download `OnlineNewsPopularity.zip`, and **unzip** it in the same directory as this ipynb file.  Then run the following cell to load the data.

In [ ]:
filename = 'OnlineNewsPopularity/OnlineNewsPopularity.csv'

# read the data
allfeatnames = []
textdata      = []
with open(filename, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        if len(allfeatnames)==0:
            allfeatnames = row
        else:
            textdata.append(row)

# put the data into a np array
dataX = empty((len(textdata), len(allfeatnames)-3))
dataY = empty(len(textdata))
for i,row in enumerate(textdata):
    # extract features (remove the first 2 features and the last feature)
    dataX[i,:] = array([float(x) for x in row[2:-1]])
    # extract target (last entry)
    dataY[i] = float(row[-1])
    
# extract feature names
featnames = [x.strip() for x in allfeatnames[2:-1]]

# extract a subset of data
dataX = dataX[::6]
dataY = dataY[::6]

print(dataX.shape)
print(dataY.shape)

There are 58 features for each article.  Here are the feature names, and an example entry.  The actual description of the features can be found in the `OnlineNewsPopularity-features.txt` file.

In [ ]:
print(featnames)

print("--- example article features---")
print(dataX[0])
print("--- example article target (# of shares)")
print(dataY[0])

Now separate the data into training and testing sets.

In [ ]:
# randomly split data into 50% train and 50% test set
trainX, testX, trainYo, testYo = \
  model_selection.train_test_split(dataX, dataY, 
  train_size=0.50, test_size=0.50, random_state=4487)

print(trainX.shape)
print(testX.shape)

Finally, we normalize the input features.

In [ ]:
# normalize feature values
# this makes comparing weights more meaningful
scaler = preprocessing.StandardScaler()  
trainXn = scaler.fit_transform(trainX)  
testXn  = scaler.transform(testX)    

Because the target value (number of shares) has a large dynamic range, we will transform the target values through the log function.

In [ ]:
# map targets to log-space
trainY = log10(trainYo)
testY  = log10(testYo)

plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
plt.hist(trainYo, 25);
plt.title('histogram of Y values')
plt.subplot(1,2,2)
plt.hist(trainY, 25);
plt.title("histogram of log(Y) values");

## 2. Prediction with Linear Regression

First we will look at predicting the number of shares using simple linear regression models.  Use the training data to fit a linear model using Ordinary Least Squares and Ridge Regression.  Use cross-validation on the training set to select the optimal $\alpha$ parameter for ridge regression, and plot the corresponding prediction accurancy on both training and testing data.

In [ ]:
### INSERT YOUR CODE HERE
## HINT: 
# 1. Ordinary Least Squares: linear_model.LinearRegression()
# 2. Ridge Regression: linear_model.Ridge(alphas= )
# 3. Rigge Regression with Cross-validation: linear_model.Ridge(alphas= )
# 4. plt.semilogx(alphas, trainAEs/testAEs)

ols = linear_model.LinearRegression()
ols.fit(trainXn,trainY)


alphas =logspace(-3,4,50)
rr = model_selection.GridSearchCV(linear_model.Ridge(),param_grid={'alpha':alphas}, cv=10, n_jobs=-1)
rr.fit(trainXn,trainY)
alpha=rr.best_params_
print(alpha)

trainAEs=[]
testAEs=[]
for alpha in alphas:
    temp_rr=linear_model.Ridge(alpha=alpha)
    temp_rr.fit(trainXn,trainY)
    trainAEs.append(metrics.mean_absolute_error(trainY, temp_rr.predict(trainXn)))
    testAEs.append(metrics.mean_absolute_error(testY, temp_rr.predict(testXn)))
plt.title("Train")
plt.semilogx(alphas, trainAEs)
   

In [ ]:
plt.title("Test")
plt.semilogx(alphas, testAEs)

Compare the two models using the _average absolute error_ (AE) between the predictions and the true values.  Below is  code that will calculate AE for the training and test sets.

In [ ]:
# ols is the linear regression model
trainAE = metrics.mean_absolute_error(trainY, ols.predict(trainXn))
testAE  = metrics.mean_absolute_error(testY, ols.predict(testXn))
print("OLS: train error =", trainAE)
print("OLS: test error =", testAE)

In [ ]:
# rr is the ridge regression model
trainAE = metrics.mean_absolute_error(trainY, rr.predict(trainXn))
testAE  = metrics.mean_absolute_error(testY, rr.predict(testXn))
print("RR: train error =", trainAE)
print("RR: test error =", testAE)

Which model has better prediction ability on the test set? Why?
- **INSERT YOUR ANSWER HERE**

ridge regression model. After normalization, ols may not detect the relation of data.

## 3. Which features are important?
Next we will investigate which features are the most important for the prediction.  Use LASSO with cross-validation to learn the model, and print the training and testing error.

In [ ]:
### INSERT YOUR CODE HERE
## HINT
# 1. LASSO with Cross-validation: linear_model.LassoCV()
las= linear_model.LassoCV(cv=5, random_state=0)
las.fit(trainXn,trainY)

trainAE = metrics.mean_absolute_error(trainY, las.predict(trainXn))
testAE  = metrics.mean_absolute_error(testY, las.predict(testXn))
print("LASSO: train error =", trainAE)
print("LASSO: test error =", testAE)

Examine the LASSO coefficients by sorting them in descending order.

In [ ]:
# sort coefficients from smallest to largest, then reverse it
inds = argsort(abs(las.coef_))[::-1]
# print out
print("weight : feature description")
for i in inds:
    print("{: .3f} : {}".format(las.coef_[i], featnames[i]))

_Which features are most important for predicting the number of shares?  For these features, which feature values (low or high values) will yield a higher number of shares?_
- **INSERT YOUR ANSWER HERE**

top 5 important features are "kw_avg_avg","LDA_02","LDA_02","data_channel_is_entertainment","average_token_length".
Higher value for "kw_avg_avg" will yield a higher number of shares.
Lower value for "LDA_02","LDA_02","data_channel_is_entertainment","average_token_length" will yield a higher number of shares.

## 3. Kernel Methods and Supprot Vector Regression 
Next, let us try some non-linear regression model such as[ kernel ridge regression](https://scikit-learn.org/stable/modules/kernel_ridge.html), [random forest regression](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) (Optional), [support vector regression](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html). To improve the prediction accuracy in Tutorial 4, and using cross-validation on the training set to select the parameters. 

In [ ]:
### INSERT YOUR CODE HERE
##Hint: 
## 1. Kernel Ridge Regression:  kernel_ridge.KernelRidge(kernel='rbf'/'poly')
## 2. Random Forest Regression: ensemble.RandomForestRegressor(random_state= , n_estimators= )
## 3. Support Vector Regression: svm.SVR(kernel='poly')
## 4. Avoid using large values of $C$ with SVR.
exps = {
    'kr-rbf': {
        'paramgrid': {'alpha': logspace(-2,3,10),'gamma': logspace(-4,3,10)},
        'clf': kernel_ridge.KernelRidge(kernel='rbf') },
    'kr-poly': {
        'paramgrid': {'alpha': logspace(-2,3,10),'gamma': logspace(-4,3,10)},
        'clf': kernel_ridge.KernelRidge(kernel='poly') },
    'rfr': {
        'paramgrid': {'n_estimators': arange(1, 20, 1), 'max_depth': arange(1, 20, 1) },
        'clf': ensemble.RandomForestRegressor() },
    'svr-poly': {
        'paramgrid': {'C': logspace(-2,3,10), 'degree': [2, 3, 4] },
        'clf': svm.SVR(kernel='poly') },
    }
clfs= {}
trainAE={}
testAE={}

In [ ]:
myname = 'kr-rbf'
clfs[myname] = model_selection.GridSearchCV(exps[myname]['clf'],param_grid=exps[myname]['paramgrid'], cv=5, n_jobs=-1)
clfs[myname].fit(trainXn,trainY)
print("best parameters for KernelRidge-RBF are:",clfs[myname].best_params_)
trainAE[myname] = metrics.mean_absolute_error(trainY, clfs[myname].predict(trainXn))
testAE[myname]  = metrics.mean_absolute_error(testY, clfs[myname].predict(testXn))
print("train error:", trainAE[myname])
print("test error :", testAE[myname])

In [ ]:
myname = 'kr-poly'
clfs[myname] = model_selection.GridSearchCV(exps[myname]['clf'],param_grid=exps[myname]['paramgrid'], cv=5, n_jobs=-1)
clfs[myname].fit(trainXn,trainY)
print("best parameters for KernelRidge-Poly are:",clfs[myname].best_params_)
trainAE[myname] = metrics.mean_absolute_error(trainY, clfs[myname].predict(trainXn))
testAE[myname]  = metrics.mean_absolute_error(testY, clfs[myname].predict(testXn))
print("train error:", trainAE[myname])
print("test error :", testAE[myname])

In [ ]:
myname = 'rfr'
clfs[myname] = model_selection.GridSearchCV(exps[myname]['clf'],param_grid=exps[myname]['paramgrid'], cv=5, n_jobs=-1)
clfs[myname].fit(trainXn,trainY)
print("best parameters for RandomForestRegressor are:",clfs[myname].best_params_)
trainAE[myname] = metrics.mean_absolute_error(trainY, clfs[myname].predict(trainXn))
testAE[myname]  = metrics.mean_absolute_error(testY, clfs[myname].predict(testXn))
print("train error:", trainAE[myname])
print("test error :", testAE[myname])

In [ ]:
myname = 'svr-poly'
clfs[myname] = model_selection.GridSearchCV(exps[myname]['clf'],param_grid=exps[myname]['paramgrid'], cv=5, n_jobs=-1)
clfs[myname].fit(trainXn,trainY)
print("best parameters for SVR are:",clfs[myname].best_params_)
trainAE[myname] = metrics.mean_absolute_error(trainY, clfs[myname].predict(trainXn))
testAE[myname]  = metrics.mean_absolute_error(testY, clfs[myname].predict(testXn))
print("train error:", trainAE[myname])
print("test error :", testAE[myname])

In [ ]:
# LASSO - poly features
polyfeats = preprocessing.PolynomialFeatures(degree=2)
trainXnf  = polyfeats.fit_transform(trainXn)
testXnf   = polyfeats.transform(testXn) 

myname = 'las-poly'
clfs[myname] = linear_model.LassoCV(max_iter=2000,tol=1)
clfs[myname].fit(trainXnf, trainY)
trainAE[myname] = metrics.mean_absolute_error(trainY, clfs[myname].predict(trainXnf))
testAE[myname]  = metrics.mean_absolute_error(testY, clfs[myname].predict(testXnf))

print("train error:", trainAE[myname])
print("test error :", testAE[myname])

In [ ]:
# sort coefficients from smallest to largest, then reverse it
inds = argsort(abs(clfs['las-poly'].coef_))[::-1]
print(len(polyfeats.powers_))
# print out
print("weight : feature description")
for i in inds:
    if abs(clfs['las-poly'].coef_[i])>1e-3:
        # get active features and powers
        pows = where(polyfeats.powers_[i])[0]
        fstr = ""
        for p in pows:
            fstr += featnames[p] + "(" + str(polyfeats.powers_[i][p]) + ") "
        print("{: .3f} : {}".format(clfs['las-poly'].coef_[i], fstr))

_Which regression method performs the best? Why do you think so?_
- **INSERT YOUR ANSWER HERE**
KernelRidge-Poly. The error is the smallest